In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# STEP 0: Install dependencies (Colab only)
!pip install folium geopy requests openpyxl

# STEP 1: Imports & API Key
import pandas as pd
import requests
import folium
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from IPython.display import FileLink
import time

TOMTOM_KEY = "YOUR-TOM-TOM-KEY"
CONGESTION_THRESHOLD = 0.6

# STEP 2: Vehicle coords (e.g., Karnal)
vehicle_coords = (28.7380,	77.1273)

# STEP 2.5: Get human-readable address using OpenStreetMap (Nominatim)
geolocator = Nominatim(user_agent="ev-route-app")
time.sleep(1)  # avoid being blocked
location = geolocator.reverse(vehicle_coords, exactly_one=True)
vehicle_address = location.address if location else "Unknown location"

# STEP 3: Load & filter your station list
file_path = "/content/drive/MyDrive/EV Project Davise Lab/ev_final.xlsx"
df = pd.read_excel(file_path)

df = df[~df["address"].str.contains("Secunderabad", na=False)]
df = df[df["city"].isin(["Delhi", "New Delhi"])]
df = df.dropna(subset=["latitude", "longitude"])
df = df.drop_duplicates(subset=["latitude", "longitude", "name"])

# STEP 4: Assign zones (you should replace this with your real logic)
df["Zone_ID_Final"] = df["city"]  # (placeholder)
zone_centroids = df.groupby("Zone_ID_Final")[["latitude", "longitude"]].mean().reset_index()

# STEP 5: Find nearest station & check congestion
df["dist_km"] = df.apply(lambda r: geodesic(vehicle_coords, (r.latitude, r.longitude)).km, axis=1)
df_sorted = df.sort_values("dist_km").reset_index(drop=True)
station = df_sorted.iloc[0]
station_coords = (station.latitude, station.longitude)

# Traffic—TomTom Flow API
cent = zone_centroids.loc[zone_centroids["Zone_ID_Final"] == station.Zone_ID_Final].iloc[0]
flow_url = (
    f"https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
    f"?point={cent.latitude},{cent.longitude}&unit=KMPH&key={TOMTOM_KEY}"
)
flow = requests.get(flow_url).json().get("flowSegmentData", {})
ratio = round(flow.get("currentSpeed", 1) / flow.get("freeFlowSpeed", 1), 2)

# Routing—TomTom Routing API
route_url = (
    f"https://api.tomtom.com/routing/1/calculateRoute/"
    f"{vehicle_coords[0]},{vehicle_coords[1]}:{station_coords[0]},{station_coords[1]}/json"
    f"?key={TOMTOM_KEY}&traffic=true&travelMode=car"
)
route_data = requests.get(route_url).json()

# STEP 6: Build Folium map
m = folium.Map(location=vehicle_coords, zoom_start=14)

# Active vehicle marker
folium.Marker(
    location=vehicle_coords,
    popup="🚗 Vehicle",
    icon=folium.Icon(color="blue", icon="car", prefix="fa")
).add_to(m)

# Nearest charging station marker
folium.Marker(
    location=station_coords,
    popup=f"🔋 {station['name']}",
    icon=folium.Icon(color="green", icon="bolt", prefix="fa")
).add_to(m)

# Draw route line if available
if "routes" in route_data and route_data["routes"]:
    summary = route_data["routes"][0]["summary"]
    route_dist = round(summary["lengthInMeters"] / 1000, 2)
    route_time = round(summary["travelTimeInSeconds"] / 60)
    points = route_data["routes"][0]["legs"][0]["points"]
    poly = [(p["latitude"], p["longitude"]) for p in points]
    folium.PolyLine(poly, color="red", weight=4, opacity=0.8).add_to(m)
else:
    print("Could not retrieve route information from TomTom API.")
    route_dist = "N/A"
    route_time = "N/A"

# STEP 7: Extract full HTML of the Folium map
map_html = m.get_root().render()

# STEP 8: Create summary panel
summary_html = f"""
<div style="font-family:Arial; max-width:700px; margin:20px auto; border:1px solid #ccc; padding:20px;">
  <h2 style="text-align:center;">🔌 EV Route Assignment</h2>
  <p><strong>📍 Vehicle Location:</strong> {vehicle_address}<br>
     <strong>Coordinates:</strong> {vehicle_coords}</p>
  <p><strong>🔋 Assigned Station:</strong> {station['name']}<br>
     <strong>Address:</strong> {station['address']}<br>
     <strong>Zone ID:</strong> {station.Zone_ID_Final}<br>
     <strong>Straight-Line Distance:</strong> {station.dist_km:.2f} km</p>
  <p><strong>🛣️ Route Distance:</strong> {route_dist} km<br>
     <strong>Estimated Time:</strong> {route_time} mins</p>
  <p><strong>🚦 Traffic Congestion Ratio:</strong> {ratio}
     {"✅ Below threshold" if ratio < CONGESTION_THRESHOLD else "⚠️ Congested zone"}</p>
</div>
"""

# STEP 9: Inject summary panel into HTML body
body_start = map_html.find("<body>") + len("<body>")
full_page = map_html[:body_start] + summary_html + map_html[body_start:]

# STEP 10: Write HTML to file
out_path = "ev_route_assignment.html"
with open(out_path, "w", encoding="utf8") as f:
    f.write(full_page)

print(f"✅ Generated {out_path}")
FileLink(out_path)


✅ Generated ev_route_assignment.html


/content/ev_route_assignment.html

In [ ]:
from IPython.core.display import HTML

html = open('ev_route_assignment.html','r',encoding='utf8').read()
display(HTML(html))

In [ ]:
from google.colab import files
files.download('ev_route_assignment.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##cd to the folder where the html file is downloaded and then run: py -3 -m http.server 8080 in the command prompt##